In [1]:
import numpy as np 
import scipy, h5py
import tables
import sys
from scipy.optimize import minimize
from numpy.polynomial import legendre as LG
import matplotlib.pyplot as plt

/home/orv/gentoo/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def Calib(theta, *args):
    total_pe, PMT_pos, cut = args
    y = total_pe
    # fixed axis
    x = Legendre_coeff(PMT_pos, *(cut,))
    # Poisson regression
    L = - np.sum(np.sum(np.transpose(y)*np.transpose(np.dot(x, theta)) \
        - np.transpose(np.exp(np.dot(x, theta)))))
    return L

In [3]:
def Legendre_coeff(PMT_pos, *args):
    cut, = args
    vertex = np.array([0,2,10,0])
    cos_theta = np.sum(vertex[1:4]*PMT_pos,axis=1)\
        /np.sqrt(np.sum(vertex[1:4]**2)*np.sum(PMT_pos**2,axis=1))
    # accurancy and nan value
    cos_theta = np.nan_to_num(cos_theta)
    cos_theta[cos_theta>1] = 1
    cos_theta[cos_theta<-1] =-1
    size = np.size(PMT_pos[:,0])
    x = np.zeros((size, cut))
    # legendre coeff
    for i in np.arange(0,cut):
        c = np.zeros(cut)
        c[i] = 1
        x[:,i] = LG.legval(cos_theta,c)
    return x  

In [4]:
def hessian(x, *args):
    total_pe, PMT_pos, cut = args
    H = np.zeros((len(x),len(x)))
    h = 1e-3
    k = 1e-3
    for i in np.arange(len(x)):
        for j in np.arange(len(x)):
            if (i != j):
                delta1 = np.zeros(len(x))
                delta1[i] = h
                delta1[j] = k
                delta2 = np.zeros(len(x))
                delta2[i] = -h
                delta2[j] = k


                L1 = - Calib(x + delta1, *(total_pe, PMT_pos, cut))
                L2 = - Calib(x - delta1, *(total_pe, PMT_pos, cut))
                L3 = - Calib(x + delta2, *(total_pe, PMT_pos, cut))
                L4 = - Calib(x - delta2, *(total_pe, PMT_pos, cut))
                H[i,j] = (L1+L2-L3-L4)/(4*h*k)
            else:
                delta = np.zeros(len(x))
                delta[i] = h
                L1 = - Calib(x + delta, *(total_pe, PMT_pos, cut))
                L2 = - Calib(x - delta, *(total_pe, PMT_pos, cut))
                L3 = - Calib(x, *(total_pe, PMT_pos, cut))
                H[i,j] = (L1+L2-2*L3)/h**2                
    return H

In [3]:
def main_Calib(Energy, radius, cut):
    filename = Energy + '/calib' + radius + '.h5'

    # read files by table
    h1 = tables.open_file(filename,'r')
    print(filename)
    truthtable = h1.root.GroundTruth
    EventID = truthtable[:]['EventID']
    ChannelID = truthtable[:]['ChannelID']
    h1.close()
    
    # read file series
    
    try:
        for j in np.arange(1,10,1):
            filename = Energy + '/calib' + radius + '_' + str(j)+ '.h5'           
            h1 = tables.open_file(filename,'r')
            print(filename)
            truthtable = h1.root.GroundTruth

            EventID_tmp = truthtable[:]['EventID']
            ChannelID_tmp = truthtable[:]['ChannelID']
            EventID = np.hstack((EventID, EventID_tmp))
            ChannelID = np.hstack((ChannelID, ChannelID_tmp))

            h1.close()
    except:
        j = j - 1
    
    total_pe = np.zeros((np.size(PMT_pos[:,0]),max(EventID)))
    for k in np.arange(1, max(EventID)):
        event_pe = np.zeros(np.size(PMT_pos[:,0]))
        hit = ChannelID[EventID == k]
        tabulate = np.bincount(hit)
        event_pe[0:np.size(tabulate)] = tabulate
        total_pe[:,k-1] = event_pe
    theta0 = np.zeros(cut) # initial value
    result = minimize(Calib,theta0, method='SLSQP',args = (total_pe, PMT_pos, cut))  
    record = np.array(result.x, dtype=float)
    
    H = hessian(result.x, *(total_pe, PMT_pos, cut))
    H_I = np.linalg.pinv(np.matrix(H))
    
    x = Legendre_coeff(PMT_pos, *(cut,))
    expect = np.mean(total_pe, axis=1)
    args = (total_pe, PMT_pos, cut)
    predict = [];
    predict.append(np.exp(np.dot(x, result.x)))
    #predict.append(expect)
    predict = np.transpose(predict)
    sum1 = 2*np.sum(- total_pe + predict + np.nan_to_num(total_pe*np.log(total_pe/predict)), axis=1)/(np.max(EventID)-30)
    
    return sum1

In [9]:
f = open(r'../PMT1t.txt')
line = f.readline()
data_list = []
while line:
    num = list(map(float,line.split()))
    data_list.append(num)
    line = f.readline()
f.close()
PMT_pos = np.array(data_list)
cut1 = 25 # Legend order
coeff = np.zeros(cut1)
for i in np.arange(cut1):
    coeff[i] = main_Calib('../0.8MeV','+0.00', i+1)/30

../0.8MeV/calib+0.00.h5


/home/orv/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/home/orv/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in log
/home/orv/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in multiply


../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5


/home/orv/gentoo/usr/lib64/python3.6/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5
../0.8MeV/calib+0.00.h5


In [16]:
coeff*30

array([31.25355463, 31.25353987, 31.25028137, 31.24988812, 31.24926721,
       31.24904264, 31.24693743, 31.24687593, 31.24667487, 31.24357998,
       31.23674118, 31.23648915, 31.23647181, 31.23497934, 31.23334958,
       31.23334958, 31.23334958, 31.23334958, 31.23334958, 31.23334958,
       31.23334958, 31.23334958, 31.23334958, 31.23334958, 31.23334958])

In [ ]:
f = open(r'../PMT1t.txt')
line = f.readline()
data_list = []
while line:
    num = list(map(float,line.split()))
    data_list.append(num)
    line = f.readline()
f.close()
PMT_pos = np.array(data_list)
cut1 = 6 # Legend order
coeff = main_Calib('../0.8MeV','+0.01', 5)

../0.8MeV/calib+0.01.h5


/home/orv/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/home/orv/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in log
/home/orv/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in multiply


In [29]:
coeff

array([1.073783  , 1.0527889 , 1.04929019, 1.05248855, 1.03930544,
       1.01708677, 1.05408382, 1.02170999, 1.05350372, 1.03664912,
       1.0367034 , 1.0595286 , 1.06443929, 1.03299461, 1.05727294,
       1.00918892, 1.05227416, 1.04243729, 1.0556442 , 1.04517507,
       1.08187887, 1.01731415, 1.01135762, 1.01257796, 1.03355807,
       1.03258785, 1.03440057, 1.06268936, 1.03832569, 1.04800231])